Installing the requirements.

In [ ]:
!pip install numpy==1.19.3
!pip install mediapipe
!pip install opencv-python
!pip install matplotlib
!pip install natsorted
!pip install pandas
!pip install plotly
!pip install seaborn

Setting up automatic reload for the scripts that will be written outside the jupyter notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

Importing all dependencies.

In [ ]:
from base64 import b64encode
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
from natsort import natsorted
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import clear_output
%matplotlib inline
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import HTML, display
import ipywidgets as widgets
from typing import List # I don't think I need this!

# Custom imports
from pose_tracking_utils import *

Setting up all the necessary variables.

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

VIDEO_PATH = "./videos/clip_training_session_2.mp4"

Now we create the pose tracking video to visualize the joints and connections.

Compress the video and visualize it on jupyter notebook.

In [ ]:
# For webcam input:
output_path = create_pose_tracking_video(VIDEO_PATH)
#compressed_path = save_compressed_video(output_path)
compressed_path = save_compressed_video(output_path)
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Create the landmark 3D plot animation.

In [ ]:
anim_output_path = f"./{VIDEO_PATH[:-4]}_anim_landmarks3D.mp4"
create_landmarks_plot3D_animation(VIDEO_PATH, anim_output_path)

compressed_path = save_compressed_video(anim_output_path)
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

The pose landmarks are this:

Another list of pose landmarks in world coordinates. Each landmark consists of the following:

x, y and z: Real-world 3D coordinates in meters with the origin at the center between hips.
visibility: Identical to that defined in the corresponding

[source](https://google.github.io/mediapipe/solutions/pose.html)

Create the reference plot against which we will set objectives to improve
the desired movements. Like for example I am using olympiam judokas
as my reference to improve my uchimata movement.

(*This function and the interactive widget are not finished*)

In [ ]:
pose_coords = get_pose_coords(video_path=VIDEO_PATH)
# remove all nones from a list
pose_coords = [x for x in pose_coords if x is not None]
print("Length of the list with the pose coordinates: ",len(pose_coords))

Now let's plot the x,y,z coordinates separately in time: 

In [ ]:
right_foot_coords = [pose_coords[i].landmark[31] for i in range(len(pose_coords))]
# Extract x, y, z coordinates from right and right foot
x_coords = [coord.x for coord in right_foot_coords]
y_coords = [coord.y for coord in right_foot_coords]
z_coords = [coord.z for coord in right_foot_coords]


# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Foot Movement Over Time')
ax.scatter(x_coords, y_coords, z_coords);

In [ ]:
# Create 3 separate plots showing each coordinates in time
fig, axs = plt.subplots(3, 1, figsize=(10, 10))
axs[0].plot(x_coords)
axs[0].set_title('X Coordinate')
axs[1].plot(y_coords)
axs[1].set_title('Y Coordinate')
axs[2].plot(z_coords)
axs[2].set_title('Z Coordinate')
plt.tight_layout();

Here we create a 3D plot animation for body parts (in this case for the feet).

In [ ]:
# x_coords = [coord.x for coord in left_foot_coords]
# y_coords = [coord.y for coord in left_foot_coords]
# z_coords = [coord.z for coord in left_foot_coords]

# x_coords_right = [coord.x for coord in right_foot_coords]
# y_coords_right = [coord.y for coord in right_foot_coords]
# z_coords_right = [coord.z for coord in right_foot_coords]

# # Create a 3D scatter plot
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('Foot Movement Over Time')

# def update(frame):
#     ax.clear()
#     ax.set_xlabel('X')
#     ax.set_ylabel('Y')
#     ax.set_zlabel('Z')
#     ax.set_title('Foot Movement Over Time')
#     ax.scatter(x_coords[:frame], y_coords[:frame], z_coords[:frame])
#     ax.scatter(x_coords_right[:frame], y_coords_right[:frame], z_coords_right[:frame])

# ani = FuncAnimation(fig, update, frames=len(x_coords), interval=5)
# ani.save('animation.mp4', writer='ffmpeg', fps=30)
# #plt.show()

Nicer looking landdmark plot of specific moments of the video, using plotly.

In [ ]:
# Getting the pose coordinates
[pose_coords[i].landmark[0].y for i in range(len(pose_coords))]
#plot_landmarks(pose_coords[200].pose_landmarks,  mp_pose.POSE_CONNECTIONS)

In [ ]:
# Creating a trace visualization
VIDEO_PATH = "./videos/uchimata_wall.mp4"
create_joint_trace(VIDEO_PATH,31, color_rgb=(0,255,0))

In [ ]:
print(VIDEO_PATH)
create_joint_trace_video(VIDEO_PATH,31, color_rgb=(255,0,0))

Realtime updating line plot of the x,y,z coordinates of the body parts.

In [ ]:
# Initialize MediaPipe Pose model
body_part_index = 32
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize OpenCV VideoCapture object to capture video from the camera
cap = cv2.VideoCapture(VIDEO_PATH)

# Create an empty list to store the trace of the right elbow
trace = []

# Create empty lists to store the x, y, z coordinates of the right elbow
x_vals = []
y_vals = []
z_vals = []

# Create a Matplotlib figure and subplot for the real-time updating plot
# fig, ax = plt.subplots()
# plt.title('Time Lapse of the X Coordinate')
# plt.xlabel('Frames')
# plt.ylabel('Coordinate Value')
# plt.xlim(0,1)
# plt.ylim(0,1)
# plt.ion()
# plt.show()
frame_num = 0

while True:
    # Read a frame from the video capture
    success, image = cap.read()
    if not success:
        break
    # Convert the frame to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose model
    results = pose.process(image)

    # Check if any body parts are detected
    
    if results.pose_landmarks:
        # Get the x,y,z coordinates of the right elbow
        x, y, z = results.pose_landmarks.landmark[body_part_index].x, results.pose_landmarks.landmark[body_part_index].y, results.pose_landmarks.landmark[body_part_index].z
        
        # Append the x, y, z values to the corresponding lists
        #x_vals.append(x)
        y_vals.append(y)
        #z_vals.append(z)
        
        # # Add the (x, y) coordinates to the trace list
        trace.append((int(x * image.shape[1]), int(y * image.shape[0])))

        # Draw the trace on the image
        for i in range(len(trace)-1):
            cv2.line(image, trace[i], trace[i+1], (255, 0, 0), thickness=2)

        plt.title('Time Lapse of the Y Coordinate')
        plt.xlabel('Frames')
        plt.ylabel('Coordinate Value')
        plt.xlim(0,len(pose_coords))
        plt.ylim(0,1)
        plt.plot(y_vals);
        # Clear the plot and update with the new x, y, z coordinate values
        #ax.clear()
        # ax.plot(range(0, frame_num + 1), x_vals, 'r.', label='x')
        # ax.plot(range(0, frame_num + 1), y_vals, 'g.', label='y')
        # ax.plot(range(0, frame_num + 1), z_vals, 'b.', label='z')
        # ax.legend(loc='upper left')
        # plt.draw()
        plt.pause(0.00000000001)
        clear_output(wait=True)
        frame_num += 1
    
    # Convert the image back to BGR format for display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Display the image
    cv2.imshow('Pose Tracking', image)

    # Wait for user input to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

# Release the video capture, close all windows, and clear the plot
cap.release()
cv2.destroyAllWindows()
plt.close()

In [ ]:
create_landmarks_plot3D_animation("videos/clip_training_session_2.mp4", 
                                  "videos/clip_training_session_2_landmarks3D.mp4")

In [16]:
def create_joint_trace_graph(video_path, output_path, body_part_index,xmin=300,xmax=1000,
                             ymin=200,ymax=800):
    """
    Creates a graph with the tracing of a particular body part,
    while executing a certain movement.
    """
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # Create an empty list to store the trace of the body part being tracked
    trace = []
    i = 0
    with mp_pose.Pose(min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                break

            # Convert the frame to RGB format
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Process the frame with MediaPipe Pose model
            results = pose.process(image)

            # Check if any body parts are detected
            if results.pose_landmarks:
                # Get the x,y coordinates of the body part being tracked (in this case, the right elbow)
                x, y = int(results.pose_landmarks.landmark[body_part_index].x * image.shape[1]), int(results.pose_landmarks.landmark[body_part_index].y * image.shape[0])

                # Add the coordinates to the trace list
                trace.append((x, y))

                # Plot the trace on the graph
                fig, ax = plt.subplots()
                #ax.imshow(image)
                ax.set_xlim(xmin,xmax)
                ax.set_ylim(ymin,ymax)
                ax.invert_yaxis()
                ax.plot(np.array(trace)[:, 0], np.array(trace)[:, 1], color='r')
                plt.savefig(f'joint_trace{i}.png')
                plt.close()
                i+=1
                #plt.pause(0.00000000001)
                #clear_output(wait=True)
                # Display the graph
                #plt.show()
            
            if cv2.waitKey(5) & 0xFF == 27:
                break
        
        cap.release()
        print("Joint Trace graph created!")
        print("Creating animation")
        create_animation_from_png(".", output_path)


video_path = "./videos/uchimata_wall.mp4"
output_path = f"{video_path[:-4]}_trace.mp4"
body_part_index = 31
create_joint_trace_graph(video_path,output_path, body_part_index)

Ignoring empty camera frame.
Joint Trace graph created!
Creating animation


In [23]:
compressed_path = save_compressed_video(output_path)
# # Show video
# mp4 = open(compressed_path,'rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML("""
# <video width=400 controls>
#       <source src="%s" type="video/mp4">
# </video>
# """ % data_url)

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/lucassoares/anaconda3/envs/explore --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './videos/uchimata_wall_trace.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512

A few ideas from ChatGPT on how to compare trackings obtained with pose tracking:

In [ ]:
#There are many ways to compare tracings in a graph when tracking body joints using pose tracking. Here are some ideas and Python code suggestions:

# 1. Euclidean distance: Calculate the Euclidean distance between each point of one tracing from the other (similar to calculating the distance between two points in math).

# ```python
import numpy as np

# Assuming "trace_a, trace_b" are two 2D numpy arrays representing the two tracings
def euclidean_distance(trace_a, trace_b):
    return np.sqrt(np.sum((trace_a - trace_b) ** 2))
```

# 2. Dynamic time warping: This is a method that measures similarity between two sequences of feature vectors. It can be used to compare two motion trajectories by measuring the cumulative distance between the corresponding points in each of the trajectories.

# ```python
from tslearn.metrics import dtw
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

# Assuming "trace_a, trace_b" are two 2D numpy arrays representing the two tracings
def dynamic_time_warping(trace_a, trace_b):
    # Normalize the traces so they have zero-mean and unit variance
    trace_a_norm = TimeSeriesScalerMeanVariance().fit_transform(trace_a)
    trace_b_norm = TimeSeriesScalerMeanVariance().fit_transform(trace_b)

    # Calculate the dynamic time warping distance between the normalized traces
    return dtw(trace_a_norm, trace_b_norm)
```

# 3. Pearson correlation coefficient: This measures the linear correlation between two variables (tracings) by calculating the covariance between them divided by the product of their standard deviations.

# ```python
from scipy.stats import pearsonr

# Assuming "trace_a, trace_b" are two 1D numpy arrays representing the two tracings
def pearson_correlation(trace_a, trace_b):
    # If the tracings have different lengths, interpolate the shorter one
    x_len, y_len = len(trace_a), len(trace_b)
    if x_len > y_len:
        trace_a = np.interp(np.linspace(0, 1, y_len), np.linspace(0, 1, x_len), trace_a)
    elif y_len > x_len:
        trace_b = np.interp(np.linspace(0, 1, x_len), np.linspace(0, 1, y_len), trace_b)

    # Calculate the Pearson correlation coefficient
    return pearsonr(trace_a, trace_b)[0]
```

# 4. Cosine distance: This measures the angle between two vectors and is commonly used to compare the similarity of two trajectories.

# ```python
from scipy.spatial.distance import cosine

# Assuming "trace_a, trace_b" are two 1D numpy arrays representing the two tracings
def cosine_distance(trace_a, trace_b):
    # If the tracings have different lengths, interpolate the shorter one
    x_len, y_len = len(trace_a), len(trace_b)
    if x_len > y_len:
        trace_a = np.interp(np.linspace(0, 1, y_len), np.linspace(0, 1, x_len), trace_a)
    elif y_len > x_len:
        trace_b = np.interp(np.linspace(0, 1, x_len), np.linspace(0, 1, y_len), trace_b)

    # Calculate the cosine distance
    return cosine(trace_a, trace_b)

#These are just a few examples of methods that can be used to compare tracings in a graph when tracking body joints using pose tracking. Depending on the specific problem you are trying to solve, there may be other methods that are more appropriate.
